<a href="https://colab.research.google.com/github/heugyu/notebook/blob/master/CNN_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
from torch.nn.functional import relu
# CNN

class CNN(nn.Module):
    def __init__(self):
        """ Initialize the model by setting up the layers"""
        super(CNN, self).__init__()

        # initial layer is resnet
        self.resnet = model.resnet18(pretrained=True, progress=False)

        # final fully connected layers
        self.dense1 = nn.Linear(1000, 500)
        self.dense2 = nn.Linear(500, 100)
        self.dense3 = nn.Linear(100, 12)

        # output layer
        self.dense4 = nn.Linear(12, 1)

    def forward(self, x):
        """ preform a forward pass of our model on some input and hidden state"""
        x = self.resnet(x)

        # apply three fully-connected Linear layers with ReLU activation function
        x = self.dense1(x)
        x = relu(x)

        x = self.dense2(x)
        x = relu(x)

        x = self.dense3(x)
        x = relu(x)

        # output is a size 1 Tensor
        x = self.dense4(x)

        return x

In [0]:
# GRU
import torch
import torch.nn as nn
from torch.nn.functional import relu
import torchvision.models as models 
from torch.utils.data import DataLoader, Dataset

class GRUnet(nn.Module):
    def __init__(self, num_features, num_rows, batch_size, hidden_size, num_layers):
        """ Initialize the model by setting up the layers """
        
        super(GRUnet, self).__init__()

        # initialize information about model
        self.num_features = num_features
        self.num_rows = num_rows
        self.batch_size = batch_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # RNN - GRU Layer
        self.rmm = nn.GRU(
            batch_first=True, 
            input_size=self.num_features,
            hidden_size=self.hidden_size,
            num_layers=self.num_layers
            )
        
        # dropout layer
        # self.dropout = nn.Dropout(0.3)

        # 3 fully-connected hidden layers - with an output of dim 1
        self.link_layer = nn.Linear(self.hidden_size, 1000)
        self.dense1 = nn.Linear(1000, 500)
        self.dense2 = nn.Linear(500, 100)
        self.dense3 = nn.Linear(100, 12)

        # output layer
        self.dense4 = nn.Linear(12, 1)

    def forward(self, x):
        """ perform a forward pass of our model on some input and hidden state """
        # GRU layer
        x, self.hidden = self.rnn(x, self.hidden)

        # detatch the hidden layer to prevent further backpropagating. i.e fix the vanishing gradient problem
        self.hideen = self.hieen.detach().cuda()

        # apply a Dropout layer
        # x = self.dropout(x)

        # pass through the link_layer
        x = self.link_layer(x)
        x = relu(x)

        # apply three fully-connected Linear layers with ReLU activation funtion
        x = self.dense1(x)
        x = relu(x)

        x = self.dense2(x)
        x = relu(x)

        x = self.dense3(x)
        x = relu(x)
        
        # output is a size 1 Tensor
        x = self.dense4(x)
        return x
    
    def init_hidden(self, batch_size, hidden_size):
        """ Initializes hidden state """

        # creates initial hidden state for GRU of zeroes
        hidden = torch.ones(self.num_layers, self.num_rows, hidden_size).cuda()
        return hidden


In [0]:
# GRU - CNN

import torch
import torch.nn as nn
from torch.nn.functional import relu
import torchvision.models as models 

class DRUCNN(nn.Module):
    def __init__(self, num_features, num_rows, batch_size, hidden_size, num_layers):
        """ Initialize the model by setting up the layers """
        super(GRUCNN, self).__init__()

        # initialize gru and cnn - the full models 

        # gru model params 
        self.num_features = num_features
        self.num_rows = num_rows
        self.batch_size = batch_size
        self.hidden_size = hidden_size
        self.n_layers = num_layers

        # resnet model
        self.cnn = models.resnet18(pretrained=True, progress=False)

        # RNN -GRU model
        self.rnn = nn.GRU(
            batch_first=True, 
            input_size=self.num_features,
            hidden_size=self.hidden_size,
            num_layers=self.num_layers
            )
        
        # init GRU hidden layer
        self.hidden = self.init_hidden(batch_size=self.batch_size, hidden_size=hidden_size)
        self.gru_output = nn.Linear(self.hidden_size, 1000)

        # final fully connected layers
        self.dense1 = nn.Linear(1000, 500)
        self.dense2 = nn.Linear(500, 100)
        self.dense3 = nn.Linear(100, 12)

        # output layer
        self.dense4 = nn.Linear(12, 1)

    def forward(self, m_input):
        """ preform a forward pass of our model on some input and hidden state """

        # input is in a tuple (gru_input, cnn_input)
        gru_input, cnn_input = m_input

        # gru
        gru_out, self.hidden = self.rnn(gru_inbput, self.hidden)
        
        # detatch the hidden layer to prevent further backpropagating. i.e. fix the vanisging gradient problem
        self.hidden = self.hidden.detach().cuda()

        # pass thriugh linear layer
        gru_out = torch.squeeze(self.gru_output(gru_out))

        # cnn
        cnn_out = self.cnn(cnn_input)

        # add the outputs of grunet and cnn
        x = gru_out.add(cnn_out)

        # feed throught final layers 
        
        # apply three fully-connected Linear layers with ReLU activection function
        x = self.dense1(x)
        x = relu(x)

        x = self.dense2(x)
        x = relu(x)

        x = self.dense3(x)
        x = relu(x)

        # output is a size 1 Tensor
        x = self.dense4(x)

        return x
    
    def init_hidden(self, batch_size, hidden_size):
        """ Initializes hidden state """

        # create initial hidden state for GRU of zeroes
        hidden = torch.ones(self.num_layers, self.batch_size, hidden_size).cuda()
        return hidden

    def load_cnn_weights(self, cnn):
        cnn_params = cnn.named_parameters()
        gru_cnn_params = dict(self.cnn.named_parameters())

        for name, cnn_param in cnn_params:
            if name in gru_cnn_params:
                gru_cnn_params[name].data.copy_(cnn_param.data)

    def load_gru_weights(self, gru):
        gru_params = gru.named_parameters()
        gru_cnn_params = dict(self.rnn.named_parameters())

        for name, gru_param in gru_params():
            if name in gru_cnn_params:
                gru_cnn_params[name].data.copy_(gru_param.data)

In [0]:

class DFDataset(Dataset):
    """ dataset with inputs of DataFrame of OCHLV + tech_ihds, ourputs of normalized np arrays """

    def __init__(self, inputs, lavels):
        self.inputs, self.labels = []. labels
        for item in inputs:
            self.inputs.append(DFDataset.format(item))
        self.c = 1 # one label

    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, i):
        return self.inputs[i], self.labels[i]
    
    @staticmethod
    def df_to_arr(df):
        return np.array(df)
    
    @staticmethod
    def normalize_df(df, norm_func):
        """ Normalize df with norm_func """
        # apply norm_func on columns
        df = df.apply(norm_func, axis=0)
        return df

    @staticmethod
    def drop_time_col(df):
        # drop time column
        return df.drop('time', axis=1)       
    
    @staticmethod
    def format(df):
        """Combine all processing steps from dataframe input to output for training"""
        # normalize with minmax
        x = DFDataset.normalize_df(df, minmaxnorm)
        x = DFDataset.drop_time_col(x)
        x = DFDataset.df_to_arr(x)
        return x   